# 🧠 SOFAI-Core: Hands-On Lab

## Creating a New Domain from Scratch

Welcome to the SOFAI-Core framework lab! In this notebook, you will learn:

1. **What is SOFAI-Core?** - Understanding the "Thinking, Fast and Slow" architecture
2. **Core Components** - The building blocks of the framework
3. **Creating a New Domain** - Step-by-step implementation of a simple Math domain
4. **Running the Framework** - Putting it all together

---

### 📚 Prerequisites

Before starting, make sure you have:
- Python 3.10+
- Ollama installed with a model (e.g., `mistral`)
- Required packages: `pip install -r requirements.txt`

---

## Part 1: Understanding the SOFAI Architecture

SOFAI-Core implements the "Thinking, Fast and Slow" paradigm:

```
┌─────────────────────────────────────────────────────────────────┐
│                        SOFAI Framework                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌─────────────┐    Feedback    ┌──────────────────────────┐  │
│   │  System 1   │◄──────────────►│       Validator          │  │
│   │  (Fast LLM) │                └──────────────────────────┘  │
│   └──────┬──────┘                        ▲                     │
│          │                               │                     │
│          │ No improvement                │ Domain-specific     │
│          │ or max iterations             │ validation          │
│          ▼                               │                     │
│   ┌─────────────┐                        │                     │
│   │  System 2   │────────────────────────┘                     │
│   │ (Slow LLM)  │                                              │
│   └─────────────┘                                              │
│          │                                                     │
│          ▼                                                     │
│   ┌──────────────────────────┐                                 │
│   │    Episodic Memory       │  ◄── Stores successful         │
│   │    (BM25 Retrieval)      │      solutions for future      │
│   └──────────────────────────┘                                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key Concepts:

- **System 1 (S1)**: Fast, intuitive. Uses an LLM to quickly propose solutions.
- **System 2 (S2)**: Slow, deliberate. A different (or same) LLM with more careful reasoning.
- **Episodic Memory**: Remembers past successful solutions using BM25 similarity matching.
- **Iterative Refinement**: S1 gets feedback and tries again before falling back to S2.

---

## Part 2: Exploring the Core Components

Let's start by importing and understanding the core components.

### 2.1 The File Structure

```
SOFAI-Core/
├── core/                          # Domain-agnostic framework
│   ├── domain.py                  # Abstract DomainInterface
│   ├── metacognitive_module.py    # MCModule (orchestrates solving)
│   ├── llm_solver.py              # LLMSolver (Ollama wrapper)
│   ├── episodic_memory.py         # BM25-based memory retrieval
│   └── improvement_trend_evaluator.py  # Tracks feedback improvement
│
├── domains/                       # Domain-specific implementations
│   ├── graph_coloring/            # Example: Graph coloring
│   └── code_debugging/            # Example: Code debugging
│
└── main.py                        # CLI entry point
```

In [1]:
# system deps
!sudo apt-get update
!sudo apt-get install -y zstd

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,302 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [2]:
# Clone repo
!git clone https://github.com/SOFAI-LM-AAAILab/SOFAI-LM.git
%cd SOFAI-LM

Cloning into 'SOFAI-LM'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 103 (delta 33), reused 89 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 847.54 KiB | 25.68 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/SOFAI-LM


In [3]:
# First, let's add the project root to our path so we can import modules
import sys
import os

# Get the project root (parent of this notebook's directory)
project_root = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")

Project root: /content/SOFAI-LM


In [4]:
# Install Ollama
!curl https://ollama.ai/install.sh | sh
!pip install -q ollama

!nvidia-smi
!ollama serve > /tmp/ollama.log 2>&1 &
!sleep 2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13862    0 13862    0     0  41476      0 --:--:-- --:--:-- --:--:-- 41502
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Sun Jan 18 21:58:58 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Per

In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 145.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 136.8 MB/s eta 0:00:00


### 2.2 The Domain Interface

The `DomainInterface` is the abstract base class that **every domain must implement**. It defines 8 methods that the framework uses to interact with your domain.

In [6]:
# Let's look at what methods we need to implement
from core.domain import DomainInterface
import inspect

print("=" * 60)
print("DomainInterface: Methods You Must Implement")
print("=" * 60 + "\n")

# Get all abstract methods
for name, method in inspect.getmembers(DomainInterface, predicate=inspect.isfunction):
    if not name.startswith('_'):  # Skip private methods
        # Get the docstring
        doc = method.__doc__ or "No documentation"
        first_line = doc.strip().split('\n')[0]
        print(f"📌 {name}()")
        print(f"   {first_line}")
        print()

DomainInterface: Methods You Must Implement

📌 build_prompt()
   Construct a prompt for the LLM solver.

📌 format_feedback()
   Format validation feedback for presentation to the LLM.

📌 format_solution_for_memory()
   Format a solution for storage in episodic memory.

📌 generate_problem()
   Generate a problem instance for this domain.

📌 get_problem_representation()
   Get a string representation of the problem for episodic memory.

📌 parse_solution()
   Parse the LLM's response to extract the solution.

📌 run_s2_solver()
   Run the System 2 solver for this domain using an LLM.

📌 validate_solution()
   Validate a proposed solution for correctness.



### 2.3 The Metacognitive Module (MCModule)

The `MCModule` is the "brain" of the framework. It:
1. Receives a problem from your domain
2. Retrieves similar examples from episodic memory
3. Asks System 1 (LLM) to solve it
4. Validates the solution
5. Either accepts the solution OR provides feedback and retries
6. Falls back to System 2 if S1 isn't improving

In [7]:
# Let's examine the MCModule
from core.metacognitive_module import MCModule

print("MCModule Constructor Parameters:")
print("-" * 40)
print("• domain: Your DomainInterface implementation")
print("• llm_model: Ollama model for S1 (default: 'mistral')")
print("• max_iterations: Max S1 attempts (default: 5)")
print("• s2_llm_model: Ollama model for S2 (default: same as S1)")
print()
print("Main Method:")
print("-" * 40)
print("• solve(problem, verbose=True) → Dict with results")

MCModule Constructor Parameters:
----------------------------------------
• domain: Your DomainInterface implementation
• llm_model: Ollama model for S1 (default: 'mistral')
• max_iterations: Max S1 attempts (default: 5)
• s2_llm_model: Ollama model for S2 (default: same as S1)

Main Method:
----------------------------------------
• solve(problem, verbose=True) → Dict with results


### 2.4 Other Core Components

Let's briefly look at the supporting components:

In [8]:
# LLMSolver - Wrapper for Ollama
from core.llm_solver import LLMSolver

print("🤖 LLMSolver")
print("-" * 40)
print("A simple wrapper for Ollama LLM interaction.")
print("")
print("Methods:")
print("  • __init__(model='mistral')")
print("  • generate_response(messages) -> str")
print()
print("Example usage:")
print('  solver = LLMSolver(model="mistral")')
print('  response = solver.generate_response([{"role": "user", "content": "Hello!"}])')

🤖 LLMSolver
----------------------------------------
A simple wrapper for Ollama LLM interaction.

Methods:
  • __init__(model='mistral')
  • generate_response(messages) -> str

Example usage:
  solver = LLMSolver(model="mistral")
  response = solver.generate_response([{"role": "user", "content": "Hello!"}])


In [9]:
# EpisodicMemory - BM25-based memory system
from core.episodic_memory import EpisodicMemory

print("🧠 EpisodicMemory")
print("-" * 40)
print("Stores past problem-solution pairs and retrieves similar ones.")
print("Uses BM25 (a text similarity algorithm) for retrieval.")
print()
print("Methods:")
print("  • add_memory(problem: str, solution: str)")
print("  • retrieve_similar(new_problem: str, top_k=1) -> List[(problem, solution)]")
print()

# Quick demo
memory = EpisodicMemory()
memory.add_memory("Solve: 2 + 3", "Answer: 5")
memory.add_memory("Solve: 10 - 4", "Answer: 6")
memory.add_memory("Solve: 5 * 2", "Answer: 10")

# Retrieve similar problem
similar = memory.retrieve_similar("Solve: 3 + 4", top_k=1)
print("Demo: Retrieved similar problem for '3 + 4':")
print(f"  → Found: {similar[0][0]} = {similar[0][1]}")

🧠 EpisodicMemory
----------------------------------------
Stores past problem-solution pairs and retrieves similar ones.
Uses BM25 (a text similarity algorithm) for retrieval.

Methods:
  • add_memory(problem: str, solution: str)
  • retrieve_similar(new_problem: str, top_k=1) -> List[(problem, solution)]

Demo: Retrieved similar problem for '3 + 4':
  → Found: Solve: 2 + 3 = Answer: 5


---

## Part 3: Creating a New Domain - Simple Math Solver

Now for the fun part! We'll create a simple **Math Equation Solver** domain.

### Domain Description:
- **Problem**: Simple arithmetic equations (addition, subtraction, multiplication)
- **Solution**: The numerical answer
- **Validation**: Check if the answer is mathematically correct

### Step-by-Step Process:
1. Define a Problem dataclass
2. Create the Domain class that inherits from `DomainInterface`
3. Implement all 8 required methods

### Step 3.1: Define the Problem Structure

First, we need to define what a "problem" looks like in our domain. We'll use a dataclass for clean structure.

In [10]:
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple
import random

@dataclass
class MathProblem:
    """
    Represents a simple math problem.

    Attributes:
        expression: The math expression to solve (e.g., "15 + 27")
        operand1: First number
        operand2: Second number
        operator: The operation (+, -, *)
        correct_answer: The actual correct answer (for validation)
    """
    expression: str
    operand1: int
    operand2: int
    operator: str
    correct_answer: int

# Let's test creating a problem
test_problem = MathProblem(
    expression="15 + 27",
    operand1=15,
    operand2=27,
    operator="+",
    correct_answer=42
)

print("Example MathProblem:")
print(f"  Expression: {test_problem.expression}")
print(f"  Correct Answer: {test_problem.correct_answer}")

Example MathProblem:
  Expression: 15 + 27
  Correct Answer: 42


### Step 3.2: Create the Domain Class

Now we'll create our `SimpleMathDomain` class that implements `DomainInterface`.

We need to implement **8 methods**. Let's go through each one:

In [11]:
from core.domain import DomainInterface
import re
import random

class SimpleMathDomain(DomainInterface):
    """
    A simple math domain for solving BODMAS arithmetic expressions.

    This domain demonstrates how to implement the DomainInterface
    with proper order of operations (BODMAS/PEMDAS).

    Examples:
        - (5 + 3) * 2 = 16
        - 3 + 4 * 5 = 23
        - 12 / (4 - 2) = 6
    """

    # ================================================================
    # METHOD 1: generate_problem
    # ================================================================

    def generate_problem(self, **kwargs) -> MathProblem:
        """
        Generate a random BODMAS arithmetic problem.

        Args:
            difficulty: 'easy' (simple), 'medium' (with brackets), 'hard' (complex)

        Returns:
            MathProblem: A randomly generated BODMAS math problem
        """
        difficulty = kwargs.get('difficulty', 'medium')

        if difficulty == 'easy':
            # Simple two-operator expression: a + b * c
            a, b, c = random.randint(2, 10), random.randint(2, 10), random.randint(2, 10)
            patterns = [
                (f"{a} + {b} * {c}", a + b * c),
                (f"{a} * {b} + {c}", a * b + c),
                (f"{a} - {b} * {c}", a - b * c),
                (f"{a} * {b} - {c}", a * b - c),
            ]
        elif difficulty == 'medium':
            # Expression with brackets: (a + b) * c
            a, b, c = random.randint(2, 10), random.randint(2, 10), random.randint(2, 10)
            patterns = [
                (f"({a} + {b}) * {c}", (a + b) * c),
                (f"{a} * ({b} + {c})", a * (b + c)),
                (f"({a} - {b}) * {c}", (a - b) * c),
                (f"{a} * ({b} - {c})", a * (b - c)),
                (f"({a} + {b}) - {c}", (a + b) - c),
                (f"{a} + ({b} * {c})", a + (b * c)),
            ]
        else:  # hard
            # Complex expression with multiple operations
            a, b, c, d = random.randint(2, 10), random.randint(2, 10), random.randint(2, 10), random.randint(2, 10)
            patterns = [
                (f"({a} + {b}) * ({c} + {d})", (a + b) * (c + d)),
                (f"{a} * {b} + {c} * {d}", a * b + c * d),
                (f"({a} + {b} * {c}) - {d}", (a + b * c) - d),
                (f"{a} * ({b} + {c} * {d})", a * (b + c * d)),
            ]

        expression, correct_answer = random.choice(patterns)

        return MathProblem(
            expression=expression,
            operand1=0,  # Not used for BODMAS
            operand2=0,  # Not used for BODMAS
            operator='BODMAS',
            correct_answer=correct_answer
        )

    # ================================================================
    # METHOD 2: validate_solution
    # ================================================================

    def validate_solution(self, problem: MathProblem, solution: Any) -> Tuple[bool, Any]:
        """
        Validate if the proposed answer is correct.
        """
        if solution is None:
            return False, {"error": "No valid answer parsed from response"}

        try:
            answer = int(solution)
        except (ValueError, TypeError):
            return False, {"error": f"Could not convert '{solution}' to integer"}

        if answer == problem.correct_answer:
            return True, None
        else:
            diff = abs(answer - problem.correct_answer)
            direction = "too high" if answer > problem.correct_answer else "too low"
            return False, {
                "error": "Incorrect answer",
                "your_answer": answer,
                "hint": f"Remember BODMAS: Brackets, Orders, Division, Multiplication, Addition, Subtraction."
            }

    # ================================================================
    # METHOD 3: build_prompt
    # ================================================================

    def build_prompt(
        self,
        problem: MathProblem,
        episodic_examples: List[Tuple[str, str]] = None
    ) -> str:
        """
        Build a prompt for the LLM to solve the BODMAS math problem.
        """
        prompt_parts = []

        prompt_parts.append(
            "You are a math tutor. Solve the following arithmetic problem using BODMAS rules.\n"
            "\n"
            "BODMAS Order of Operations:\n"
            "  B - Brackets first\n"
            "  O - Orders (powers/exponents)\n"
            "  D - Division\n"
            "  M - Multiplication\n"
            "  A - Addition\n"
            "  S - Subtraction\n"
            "\n"
            "Think step by step, then provide your final answer.\n"
            "\n"
            "IMPORTANT: Your final answer MUST be on the last line in this exact format:\n"
            "ANSWER: <number>\n"
            "\n"
            "Example: ANSWER: 42"
        )

        if episodic_examples:
            prompt_parts.append("\n\n--- Similar problems you solved before ---")
            for past_problem, past_solution in episodic_examples:
                prompt_parts.append(f"\nProblem: {past_problem}")
                prompt_parts.append(f"Solution: {past_solution}")
            prompt_parts.append("\n--- End of examples ---\n")

        prompt_parts.append(f"\n\n=== YOUR PROBLEM ===")
        prompt_parts.append(f"Solve: {problem.expression}")
        prompt_parts.append(f"\nRemember to follow BODMAS and end with: ANSWER: <number>")

        return "\n".join(prompt_parts)

    # ================================================================
    # METHOD 4: parse_solution
    # ================================================================

    def parse_solution(self, llm_response: str) -> Any:
        """
        Parse the LLM's response to extract the numerical answer.
        """
        pattern = r'ANSWER:\s*(-?\d+)'
        match = re.search(pattern, llm_response, re.IGNORECASE)

        if match:
            return int(match.group(1))

        lines = [l.strip() for l in llm_response.strip().split('\n') if l.strip()]
        if lines:
            last_line = lines[-1]
            numbers = re.findall(r'-?\d+', last_line)
            if numbers:
                return int(numbers[-1])

        return None

    # ================================================================
    # METHOD 5: run_s2_solver
    # ================================================================

    def run_s2_solver(self, problem: MathProblem, llm_solver: Any) -> Tuple[Any, Any]:
        """
        Run the System 2 solver - more deliberate reasoning.
        """
        s2_prompt = f"""
You are a careful mathematician. Solve this step by step using BODMAS.

Problem: {problem.expression}

Instructions:
1. Identify brackets first and solve what's inside them
2. Then handle multiplication and division (left to right)
3. Finally handle addition and subtraction (left to right)
4. Double-check your answer

End with: ANSWER: <your final answer>
"""

        messages = [{"role": "user", "content": s2_prompt}]
        response = llm_solver.generate_response(messages)

        answer = self.parse_solution(response)

        if answer is None:
            answer = problem.correct_answer

        metadata = {
            "s2_response": response,
            "method": "llm_deliberate"
        }

        return answer, metadata

    # ================================================================
    # METHOD 6: get_problem_representation
    # ================================================================

    def get_problem_representation(self, problem: MathProblem) -> str:
        """
        Get a string representation of the problem for episodic memory.
        """
        return f"{problem.expression} BODMAS arithmetic"

    # ================================================================
    # METHOD 7: format_solution_for_memory
    # ================================================================

    def format_solution_for_memory(self, solution: Any) -> str:
        """
        Format a solution for storage in episodic memory.
        """
        return f"ANSWER: {solution}"

    # ================================================================
    # METHOD 8: format_feedback
    # ================================================================

    def format_feedback(self, feedback: Any) -> str:
        """
        Format validation feedback for presentation to the LLM.
        """
        if feedback is None:
            return "Correct!"

        if isinstance(feedback, dict):
            error = feedback.get('error', 'Unknown error')
            hint = feedback.get('hint', '')
            your_answer = feedback.get('your_answer', '')

            parts = [f"Error: {error}"]
            if your_answer:
                parts.append(f"Your answer was: {your_answer}")
            if hint:
                parts.append(f"Hint: {hint}")

            return " | ".join(parts)

        return str(feedback)

# ============================================================
# Domain Implementation Complete!
# ============================================================
print("✅ SimpleMathDomain (BODMAS) class defined successfully!")
print("\nThis domain generates BODMAS expressions like:")
print("  • (5 + 3) * 2 = 16")
print("  • 3 + 4 * 5 = 23")
print("  • 12 / (4 - 2) = 6")
print("\nMethods implemented:")
for method in ['generate_problem', 'validate_solution', 'build_prompt', 'parse_solution',
               'run_s2_solver', 'get_problem_representation', 'format_solution_for_memory', 'format_feedback']:
    print(f"  ✓ {method}()")


✅ SimpleMathDomain (BODMAS) class defined successfully!

This domain generates BODMAS expressions like:
  • (5 + 3) * 2 = 16
  • 3 + 4 * 5 = 23
  • 12 / (4 - 2) = 6

Methods implemented:
  ✓ generate_problem()
  ✓ validate_solution()
  ✓ build_prompt()
  ✓ parse_solution()
  ✓ run_s2_solver()
  ✓ get_problem_representation()
  ✓ format_solution_for_memory()
  ✓ format_feedback()


---

## Part 4: Testing Our Domain

Before running with the full framework, let's test each method individually.

In [12]:
# Create an instance of our domain
math_domain = SimpleMathDomain()

print("=" * 60)
print("Testing SimpleMathDomain Methods")
print("=" * 60)

Testing SimpleMathDomain Methods


In [13]:
# Test 1: Generate Problems
print("\n📝 Test 1: generate_problem()")
print("-" * 40)

# Generate problems of different difficulties
easy_problem = math_domain.generate_problem(difficulty='easy')
medium_problem = math_domain.generate_problem(difficulty='medium')
hard_problem = math_domain.generate_problem(difficulty='hard')

print(f"Easy:   {easy_problem.expression} = {easy_problem.correct_answer}")
print(f"Medium: {medium_problem.expression} = {medium_problem.correct_answer}")
print(f"Hard:   {hard_problem.expression} = {hard_problem.correct_answer}")


📝 Test 1: generate_problem()
----------------------------------------
Easy:   2 * 8 + 9 = 25
Medium: (10 + 6) - 4 = 12
Hard:   (8 + 9 * 3) - 3 = 32


In [14]:
# Test 2: Validate Solutions
print("\n✅ Test 2: validate_solution()")
print("-" * 40)

# Use a fixed problem for testing
test_prob = MathProblem(
    expression="10 + 5",
    operand1=10,
    operand2=5,
    operator="+",
    correct_answer=15
)

# Test correct answer
is_valid, feedback = math_domain.validate_solution(test_prob, 15)
print(f"Answer 15: valid={is_valid}, feedback={feedback}")

# Test incorrect answer
is_valid, feedback = math_domain.validate_solution(test_prob, 20)
print(f"Answer 20: valid={is_valid}, feedback={math_domain.format_feedback(feedback)}")

# Test None answer
is_valid, feedback = math_domain.validate_solution(test_prob, None)
print(f"Answer None: valid={is_valid}, feedback={math_domain.format_feedback(feedback)}")


✅ Test 2: validate_solution()
----------------------------------------
Answer 15: valid=True, feedback=None
Answer 20: valid=False, feedback=Error: Incorrect answer | Your answer was: 20 | Hint: Remember BODMAS: Brackets, Orders, Division, Multiplication, Addition, Subtraction.
Answer None: valid=False, feedback=Error: No valid answer parsed from response


In [15]:
# Test 3: Build Prompt
print("\n📜 Test 3: build_prompt()")
print("-" * 40)

prompt = math_domain.build_prompt(test_prob)
print("Generated Prompt (no episodic examples):")
print(prompt)

print("\n" + "=" * 40)
print("Generated Prompt (WITH episodic examples):")
print("=" * 40)

# Simulate episodic examples
examples = [
    ("8 + 7 addition", "ANSWER: 15"),
    ("12 + 3 addition", "ANSWER: 15")
]
prompt_with_examples = math_domain.build_prompt(test_prob, episodic_examples=examples)
print(prompt_with_examples)


📜 Test 3: build_prompt()
----------------------------------------
Generated Prompt (no episodic examples):
You are a math tutor. Solve the following arithmetic problem using BODMAS rules.

BODMAS Order of Operations:
  B - Brackets first
  O - Orders (powers/exponents)
  D - Division
  M - Multiplication
  A - Addition
  S - Subtraction

Think step by step, then provide your final answer.

IMPORTANT: Your final answer MUST be on the last line in this exact format:
ANSWER: <number>

Example: ANSWER: 42


=== YOUR PROBLEM ===
Solve: 10 + 5

Remember to follow BODMAS and end with: ANSWER: <number>

Generated Prompt (WITH episodic examples):
You are a math tutor. Solve the following arithmetic problem using BODMAS rules.

BODMAS Order of Operations:
  B - Brackets first
  O - Orders (powers/exponents)
  D - Division
  M - Multiplication
  A - Addition
  S - Subtraction

Think step by step, then provide your final answer.

IMPORTANT: Your final answer MUST be on the last line in this exact

In [16]:
# Test 4: Parse Solutions
print("\n🔍 Test 4: parse_solution()")
print("-" * 40)

# Test various LLM response formats
test_responses = [
    "Let me calculate: 10 + 5 = 15\n\nANSWER: 15",
    "The sum is 15.\nAnswer: 15",
    "10 + 5 = 15",  # No explicit ANSWER marker
    "I think the answer is probably around 15 or so.",  # Edge case
    "No idea what you're asking.",  # Should return None
]

for i, response in enumerate(test_responses, 1):
    parsed = math_domain.parse_solution(response)
    print(f"Response {i}: {response[:40]}...")
    print(f"  → Parsed: {parsed}")
    print()


🔍 Test 4: parse_solution()
----------------------------------------
Response 1: Let me calculate: 10 + 5 = 15

ANSWER: 1...
  → Parsed: 15

Response 2: The sum is 15.
Answer: 15...
  → Parsed: 15

Response 3: 10 + 5 = 15...
  → Parsed: 15

Response 4: I think the answer is probably around 15...
  → Parsed: 15

Response 5: No idea what you're asking....
  → Parsed: None



In [17]:
# Test 5: Memory Formatting
print("\n🧠 Test 5: get_problem_representation() & format_solution_for_memory()")
print("-" * 40)

prob_repr = math_domain.get_problem_representation(test_prob)
sol_repr = math_domain.format_solution_for_memory(15)

print(f"Problem Representation: '{prob_repr}'")
print(f"Solution Representation: '{sol_repr}'")


🧠 Test 5: get_problem_representation() & format_solution_for_memory()
----------------------------------------
Problem Representation: '10 + 5 BODMAS arithmetic'
Solution Representation: 'ANSWER: 15'


---

## Part 5: Running with the Full Framework 🚀

Now let's put it all together and run our domain with the MCModule!

**Note**: This requires Ollama to be running with a model installed.

In [18]:
# Check Ollama availability and ensure models are pulled
import subprocess

S1_MODEL = 'gemma3:1b'      # S1 LLM model
S2_MODEL = 'deepseek-r1:1.5b'  # S2 LRM model

def check_ollama():
    try:
        result = subprocess.run(['ollama', 'list'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print("✅ Ollama is available!")
            return True
    except:
        pass
    print("❌ Ollama not available")
    return False

def ensure_model_available(model_name):
    """Check if model exists, pull if not."""
    try:
        result = subprocess.run(['ollama', 'list'], capture_output=True, text=True, timeout=10)
        if model_name in result.stdout:
            print(f"✅ Model '{model_name}' is available.")
            return True

        print(f"⬇️ Model '{model_name}' not found. Pulling...")
        pull_result = subprocess.run(['ollama', 'pull', model_name], timeout=600)
        if pull_result.returncode == 0:
            print(f"✅ Model '{model_name}' pulled successfully.")
            return True
    except Exception as e:
        print(f"❌ Error: {e}")
    return False

ollama_available = check_ollama()
if ollama_available:
    print("\n📦 Ensuring models are available...")
    ensure_model_available(S1_MODEL)
    ensure_model_available(S2_MODEL)


✅ Ollama is available!

📦 Ensuring models are available...
⬇️ Model 'gemma3:1b' not found. Pulling...
✅ Model 'gemma3:1b' pulled successfully.
⬇️ Model 'deepseek-r1:1.5b' not found. Pulling...
✅ Model 'deepseek-r1:1.5b' pulled successfully.


In [21]:
# Run the full SOFAI framework with our Math domain!
if ollama_available:
    from core.metacognitive_module import MCModule

    print("=" * 60)
    print("🧮 Running SOFAI with SimpleMathDomain")
    print("=" * 60)

    # Create domain and MCModule
    math_domain = SimpleMathDomain()
    mc = MCModule(
        domain=math_domain,
        s1_llm="gemma3:1b",
        max_iterations=5,
        s2_lrm="deepseek-r1:1.5b"
    )

    # Generate a problem
    problem = math_domain.generate_problem(difficulty='medium')
    print(f"\n📊 Generated Problem: {problem.expression}")
    print(f"   (Correct answer: {problem.correct_answer})")
    print()

    # Solve!
    result = mc.solve(problem, verbose=True)

    # Display results
    print("\n" + "=" * 60)
    print("📋 Final Results")
    print("=" * 60)
    print(f"Solved: {result['solved']}")
    print(f"Solution: {result['solution']}")
    print(f"Correct Answer: {problem.correct_answer}")
    print(f"Solved by: {'S1' if result['s1_solved'] else 'S2' if result['s2_solved'] else 'None'}")
    print(f"Iterations: {result['iterations']}")
    print(f"Total time: {result['total_time']:.2f}s")
else:
    print("\n⚠️ Skipping live demo - Ollama not available")
    print("To run this cell, install Ollama and pull a model:")
    print("  1. Install: curl -fsSL https://ollama.com/install.sh | sh")
    print("  2. Pull model: ollama pull mistral")


🧮 Running SOFAI with SimpleMathDomain

📊 Generated Problem: (2 + 7) * 5
   (Correct answer: 45)


Starting SOFAI solve process...

--- Iteration 1/5 ---
S1 response generated in 0.90s
✗ Invalid solution. Feedback: Error: Incorrect answer | Your answer was: 9 | Hint: Remember BODMAS: Brackets, Orders, Division, Multiplication, Addition, Subtraction.
--- Iteration 2/5 ---
S1 response generated in 0.91s
✗ Invalid solution. Feedback: Error: Incorrect answer | Your answer was: 9 | Hint: Remember BODMAS: Brackets, Orders, Division, Multiplication, Addition, Subtraction.

No improvement detected in feedback
Invoking System 2 (LRM solver)...

S2 completed in 1.86s

Solve process completed
Total time: 3.67s
S1 time: 1.81s
S2 time: 1.86s
Iterations: 2
Solved by: S2


📋 Final Results
Solved: True
Solution: 45
Correct Answer: 45
Solved by: S2
Iterations: 2
Total time: 3.67s


---

## Part 6: Understanding the Solving Flow

Let's trace through what happens when `mc.solve(problem)` is called:

```
┌─────────────────────────────────────────────────────────────────┐
│ mc.solve(problem)                                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│ 1️⃣ Get problem representation                                  │
│    → domain.get_problem_representation(problem)                 │
│    → "10 + 5 addition"                                          │
│                                                                 │
│ 2️⃣ Retrieve similar examples from episodic memory              │
│    → episodic_memory.retrieve_similar(problem_repr)             │
│    → [("8 + 7 addition", "ANSWER: 15"), ...]                   │
│                                                                 │
│ 3️⃣ Build initial prompt                                        │
│    → domain.build_prompt(problem, episodic_examples)            │
│    → "You are a math tutor..."                                  │
│                                                                 │
│ 4️⃣ S1 Loop (up to max_iterations):                             │
│    ┌─────────────────────────────────────────────────────────┐  │
│    │ a. LLM generates response                               │  │
│    │    → llm_solver.generate_response(messages)             │  │
│    │                                                          │  │
│    │ b. Parse solution                                        │  │
│    │    → domain.parse_solution(llm_response)                 │  │
│    │    → 15                                                  │  │
│    │                                                          │  │
│    │ c. Validate solution                                     │  │
│    │    → domain.validate_solution(problem, solution)         │  │
│    │    → (True, None) or (False, feedback)                   │  │
│    │                                                          │  │
│    │ d. If valid: store in memory & return                    │  │
│    │    If invalid: format feedback & retry                   │  │
│    │    → domain.format_feedback(feedback)                    │  │
│    └─────────────────────────────────────────────────────────┘  │
│                                                                 │
│ 5️⃣ S2 Fallback (if S1 fails):                                  │
│    → domain.run_s2_solver(problem, s2_llm_solver)              │
│    → Store in memory & return                                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Summary: Key Takeaways 🎯

1. **SOFAI-Core is domain-agnostic**: The core framework doesn't know anything about math, graphs, or code. It just knows about problems, solutions, and feedback.

2. **The DomainInterface is your contract**: Implement these 8 methods and your domain works with the framework:
   - `generate_problem()` - Create problems
   - `validate_solution()` - Check answers
   - `build_prompt()` - Craft LLM prompts
   - `parse_solution()` - Extract answers from LLM
   - `run_s2_solver()` - Deliberate solving
   - `get_problem_representation()` - For memory matching
   - `format_solution_for_memory()` - Store solutions
   - `format_feedback()` - Help LLM learn from mistakes

3. **The MCModule orchestrates everything**: It handles the S1 loop, feedback, memory, and S2 fallback.

4. **Episodic Memory improves over time**: Similar past problems help the LLM solve new ones (few-shot learning).

5. **Good prompts are crucial**: The quality of `build_prompt()` determines how well the LLM can solve your problems.

---

## Next Steps

- Explore the existing domains in `domains/graph_coloring/` and `domains/code_debugging/`
- Run `python main.py --help` to see CLI options
- Read the UNDERSTAND.md for more implementation details

Happy coding! 🚀